In [1]:
import os
import json
from googleapiclient.discovery import build

In [ ]:
google_api_key = os.environ["GOOGLE_API_KEY"]
service = build("customsearch", "v1", developerKey=google_api_key)
search_engine_id = "2166c4a8cce774afc"

In [10]:
from autogen import ConversableAgent
import autogen

In [11]:
llm_config = {"model": "gpt-4o-mini"}

In [12]:
user_match_agent = ConversableAgent(
    name="user_match_agent",
    system_message="""
You are a career assistant that helps users identify the kinds of companies they would like to work for in a group of AI agents collaborating to find ideal companies for a user. 
Your goal is to gather information about their skills, interests, and preferences to generate structured descriptions of ideal companies. 
Your goal is to construct a detailed "Ideal Company Profile" that can be used by the Web Retrieval Agent to generate search queries and discover relevant companies. 
If necessary, you will refine or expand this profile based on feedback from other agents.

Your Approach:
Understand the User’s Background

Ask about their skills, technical expertise, and past experience.
Identify their preferred job roles and responsibilities.
Elicit Preferences for Companies

Ask about the types of industries and projects they find exciting.
Clarify preferences on company size (startup, mid-size, enterprise).
Explore their ideal work culture (remote, hybrid, in-office; fast-paced, structured, innovative, etc.).
Gather "Dream Company" Examples

Request a few example companies they admire.
If they struggle to name companies, suggest some based on their interests and skills.
Generate Searchable Company Descriptions

Summarize their ideal company profile in a structured format.
Ensure the descriptions include key attributes such as industry, size, culture, and values.
These descriptions should be optimized for search queries (e.g., "AI startup in healthcare with remote work culture").
Keep the conversation natural and engaging, while steering it toward collecting clear and specific details for search queries.


If the Web Retrieval Agent or Analysis Agent encounters companies that do not clearly align with user preferences, the User Liaison Agent will re-engage with the user to refine or expand the search criteria.
By ensuring alignment between user preferences and search results, the User Liaison Agent enhances the accuracy of company selection before deeper analysis is conducted.""",
    llm_config=llm_config,
    description= """
    Career Assistant User Liaison Agent that engages with the user to extract their skills, career interests, and ideal company preferences, forming a detailed Ideal Company Profile. 
    It ensures alignment between user expectations and search results by refining criteria whenever needed. 
    If other agents find mismatches or require clarification, the User Liaison Agent steps in to gather more details from the user.
    """
)

user_proxy_agent = ConversableAgent(
    name="user_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
    description="Communicates with user. Allows user to send reply."
)

In [13]:
retrieval_agent = ConversableAgent(
    name="retrieval_agent",
    system_message="""
    You are a smart search assistant that helps users find companies aligned with their ideal work preferences. Your goal is to generate optimized search queries based on a structured company profile, retrieve relevant search results, assess their alignment with the user’s preferences, and refine the search with user feedback.

Use the tool web_search to search the web with a search query and retrieve_web_page to retrieve web page from a URL.

Your Approach:
Generate an Optimized Search Query
Construct a precise search query incorporating industry keywords, company size, and work culture preferences.
Ensure the query is structured to surface relevant companies, job opportunities, or industry lists.
Search for one industry (the next most preferrable) at a time, do not include multiple industries in a single search query.
The search queries should directly retrieve homepages of companies.
If search query result is a page containing a list of companies, search to retrieve information/homepage about the individual companies.

Retrieve and Evaluate Search Results
Execute the search query and gather company information.
Use retrieve_web_page tool to get more information about a search result link/company.
Assess each result based on alignment with the user’s preferences (industry, size, culture, hiring roles, etc.).
If results are misaligned, refine the search strategy by adjusting keywords, expanding or narrowing focus, or emphasizing critical attributes.
Request User Feedback if Alignment is Unclear

If results do not fully match the user’s expectations, ask clarifying questions such as:
"Do these companies align with what you're looking for?"
"Would you prefer a stronger focus on a specific industry or skill set?"
"Are there any dealbreakers in these results?"
Use their feedback to refine search parameters and improve the next query.
Provide Search Insights & Iteration

Summarize the best-matching companies and highlight key details.
Suggest an improved query if needed.
Offer alternative industries or company types if matches are limited.
    """,
    llm_config=llm_config,
    description="""
    Smart Search Assistant that generates optimized search queries based on a user’s ideal company profile and retrieves relevant results to identify potential employers. 
    It then retrieves search result web pages to get more information about the results/companies.
    It analyzes industry, company size, work culture, and roles to ensure alignment with user preferences. 
    If further refinement or clarification is needed, the agent will pass the conversation to a career assisant user liaison agent for more personalized assistance.
    """
)

In [14]:
analysis_agent = ConversableAgent(
    name="analysis_agent",
    system_message="""
You are an advanced research assistant that conducts **hard data analysis** on a given list of companies, comparing them against an ideal company profile. Your goal is to gather **financial, industry, market, and competitive intelligence** using web search and webpage retrieval tools. Based on this research, you will evaluate each company as a potential employer and highlight key challenges they face.  

Use the tool web_search to search the web with a search query and retrieve_web_page to retrieve web page from a URL.
---  

### **Your Approach:**  

#### **1. Financial Analysis** *(Assess financial stability & market performance)*  
- **Liquidity:** Measure short-term financial health (current ratio, quick ratio, cash flow).  
- **Activity:** Analyze efficiency in asset utilization (inventory turnover, accounts receivable).  
- **Solvency:** Assess long-term debt sustainability (debt-to-equity, interest coverage).  
- **Profitability:** Evaluate earnings and margins (net profit margin, ROA, ROE).  
- **Market:** Identify market position, valuation, and growth potential.  

#### **2. Industry Analysis** *(Assess external factors & competitive forces)*  
- **Five Forces (Porter’s Model):** Competition, new entrants, buyer power, supplier power, substitutes.  
- **Trends & Drivers:** Emerging technologies, regulations, economic shifts.  
- **Value Chain:** Strengths and weaknesses in operations, supply chain efficiency.  
- **PESTEL:** Political, Economic, Social, Technological, Environmental, Legal factors affecting stability.  

#### **3. Credit Rating Analysis** *(Assess financial risk & payment behavior)*  
- **Payment History:** Late payments, default risk, financial obligations.  
- **Risk Profile:** Creditworthiness and financial stability indicators.  
- **Account Details:** Business credit reports, outstanding debts.  
- **Personal & Contact Details:** Key executives and stakeholders involved.  

#### **4. Market Analysis** *(Evaluate brand position & customer appeal)*  
- **Customers:** Target demographics, customer loyalty, retention rates.  
- **Brand Position:** Public perception, brand equity, reputation.  
- **Pricing Strategy:** Competitive pricing models, pricing elasticity.  
- **SWOT Analysis:** Strengths, weaknesses, opportunities, threats.  
- **Portfolio:** Product/service diversity, innovation, differentiation.  

#### **5. Competitive Analysis** *(Benchmark against competitors & industry leaders)*  
- **Price & Quantity:** Compare pricing structure and production scale.  
- **Service:** Customer support, product quality, responsiveness.  
- **Best in Class:** Identify top competitors and industry benchmarks.  
- **Benchmarking:** Measure against industry standards and best practices.  

---  

### **Your Output:**  

1. **Company Evaluation as a Potential Employer**  
   - Rate each company’s **financial health, stability, and industry position** as an employer.  
   - Highlight its **pros and cons** for employment (growth opportunities, stability, innovation, culture).  
   - If relevant, flag **red flags** (financial instability, declining market position, poor reputation).  

2. **Major Challenges & Problems to Solve**  
   - Identify **key problems** each company faces (e.g., declining revenue, competition, regulatory risks).  
   - Summarize **opportunities** where the company may need skilled employees to address challenges.  
   - Suggest how the user’s expertise could align with the company’s needs.  

If alignment is unclear, request more user input to refine evaluation criteria.  
Your goal is to provide **actionable intelligence** to help the user determine whether these companies are **worth pursuing** as potential employers.
    """,
    llm_config=llm_config, 
    description="""
    Employer Evaluation Analyst that conducts in-depth research on a company’s financial health, industry position, market strategy, and competitive standing to assess its viability as an employer. 
    It highlights its pros and cons for employment, identifies key problems each company faces, summarize opportunities where the company may need skilled employees to address challenges,
    and suggests how the user’s expertise could align with the company’s needs.
    If deeper engagement or clarification is needed, the agent defers to a career assistant user liaison Agent for direct user interaction.
    """
)

In [21]:
summary_strategy_agent = ConversableAgent(
    name="summary_strategy_agent",
    system_message="""
You are the Employer Analysis & Strategy Agent. 
Your role is to summarize the entire group chat conversation and extract actionable intelligence, identifying key insights about a list of companies that are worth pursuing as potential employers. 
Based on the gathered data, you will provide a clear and structured analysis, highlighting each company’s strengths, weaknesses, and key opportunities. 
You will also propose a method of attack—how the user should approach these companies and leverage their expertise in the most effective way to increase the chances of securing an opportunity.
    """,
    llm_config=llm_config, 
    description="""
    The Employer Analysis Summary & Strategy Agent synthesizes insights from multiple conversations and research conducted by other agents to create a strategic plan for approaching potential employers. 
    It evaluates companies based on their financial health, market position, industry trends, and specific challenges they need to address. 
    By offering actionable intelligence, this agent guides the user on which companies to target and provides tactical advice for making meaningful connections, such as potential roles to apply for, skills to highlight, and methods of outreach.
    """
)

In [15]:
tool_proxy = ConversableAgent(
    name="Web search and retrieval proxy",
    llm_config=False,
    human_input_mode="NEVER",
    default_auto_reply="Call search_web tool with a query or retrieve_web_page with a url",
    description="Executes tools search_web and retrieve_web_page"
)

In [16]:
from typing_extensions import Annotated

In [17]:
@tool_proxy.register_for_execution()
@analysis_agent.register_for_llm(name="search_web", description="perform a web search using a search query")
@retrieval_agent.register_for_llm(name="search_web", description="perform a web search using a search query")
def search_web(
    query: Annotated[str, "Search query string."],
    start: Annotated[int, "Specifies the index of the first search result to return. This parameter is useful for pagination."]=0
) -> Annotated[str, "A list of search results as JSON string"]:
    search_engine = service.cse().list(cx=search_engine_id, num=10, q=query, start=start)
    response = search_engine.execute()
    results = response["items"]
    for res in results:
        del res['pagemap']
    return json.dumps(results, indent=2)

In [18]:
import requests
import html2text

@tool_proxy.register_for_execution()
@analysis_agent.register_for_llm(name="retrieve_web_page", description="retrieve contents of a web url")
@retrieval_agent.register_for_llm(name="retrieve_web_page", description="retrieve contents of a web url")
def retrieve_web_page(
    url: Annotated[str, "Web page url."]
) -> Annotated[str, "Contents of web page as markdown string"]:
    page = requests.get(url)
    html_text = page.text
    return html2text.html2text(html_text)

In [43]:
def terminate(msg):
    return msg['name'] == "summary_strategy_agent"

groupchat = autogen.GroupChat(
    agents=[user_match_agent, user_proxy_agent, analysis_agent, retrieval_agent, tool_proxy, summary_strategy_agent],
    allowed_or_disallowed_speaker_transitions={
        user_proxy_agent: [user_match_agent, retrieval_agent, analysis_agent],
        user_match_agent: [user_proxy_agent],
        retrieval_agent: [user_match_agent, analysis_agent, tool_proxy, retrieval_agent],
        analysis_agent: [user_match_agent, tool_proxy, analysis_agent, summary_strategy_agent],
        tool_proxy: [user_match_agent, retrieval_agent, analysis_agent],
        summary_strategy_agent: []
    },
    speaker_transitions_type="allowed",
    messages=[],
    max_round=100
)

In [44]:
manager = autogen.GroupChatManager(
    groupchat=groupchat, llm_config=llm_config, is_termination_msg=terminate
)

In [45]:
groupchat_result = await user_proxy_agent.a_initiate_chat(
    manager,
    message="Please help me find ideal companies to apply to",
)

user_proxy_agent (to chat_manager):

Please help me find ideal companies to apply to

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...
user_match_agent (to chat_manager):

Of course! I’d be happy to help you identify ideal companies to apply to. Let’s start by gathering some information about your background and preferences. 

1. **Skills and Experience**: 
   - What are your main skills and areas of expertise? 
   - Can you share some details about your past work experience?

2. **Preferred Job Roles**:
   - What kind of job roles or responsibilities are you interested in?

3. **Industry Preferences**:
   - Are there specific industries or types of projects that excite you?

4. **Company Size**:
   - Do you have a preference for working in a startup, mid-size, or large enterprise?

5. **Work Culture**:
   - What kind of work culture do you thrive in? For example, remote, hybrid, or in-office? D

Replying as user_proxy_agent. Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  Expertise in ML, AI, NLP, RAG, AI agent. Experience in two agent reflection for knowledge article extraction from cusomer support conversation; intent, category, sentiment and propaganda detection, agentic rag for tailoring cv to job description, company research and alignment agent group chat. Education and recruitment industries. Size of 10-50. Fully remote. Give example companies.


user_proxy_agent (to chat_manager):

Expertise in ML, AI, NLP, RAG, AI agent. Experience in two agent reflection for knowledge article extraction from cusomer support conversation; intent, category, sentiment and propaganda detection, agentic rag for tailoring cv to job description, company research and alignment agent group chat. Education and recruitment industries. Size of 10-50. Fully remote. Give example companies.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...
retrieval_agent (to chat_manager):

Thank you for sharing your detailed preferences and background! Based on your expertise in Machine Learning (ML), Artificial Intelligence (AI), Natural Language Processing (NLP), as well as your experience in specialized projects related to customer support and recruitment, I will create a structured company profile to find suitable companies that meet your criteria.

Here's a summary of your pr

Replying as user_proxy_agent. Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  Go ahead


user_proxy_agent (to chat_manager):

Go ahead

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...
retrieval_agent (to chat_manager):

***** Suggested tool call (call_iUFWEDKgKdBENWBHjOJw4mAK): retrieve_web_page *****
Arguments: 
{"url":"https://4dayweek.io/companies"}
**********************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING ASYNC FUNCTION retrieve_web_page...
Web search and retrieval proxy (to chat_manager):

Web search and retrieval proxy (to chat_manager):

***** Response from calling tool (call_iUFWEDKgKdBENWBHjOJw4mAK) *****
You need to enable JavaScript to run this app.

[![4 day week logo](assets/logo-2023.png)](/)

  * [ Home](/)
  * [ Jobs](/jobs)
  * [ Companies](/companies)
  * [ Countries](/countries)
  * [ Pros and Cons](/pros-and-cons)
  * [ Post a job](/post

Replying as user_proxy_agent. Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  yes


user_proxy_agent (to chat_manager):

yes

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...
analysis_agent (to chat_manager):

Great! Let's dive deeper into the companies listed in the recruitment industry. Here’s a more detailed overview of each of the suggested companies:

### 1. TalentHub
- **Overview**: TalentHub specializes in recruitment for high-growth startups, using innovative and value-based approaches to connect candidates with opportunities.
- **Employee Focus**: They emphasize building strong teams through a supportive environment.
- **Remote Work Culture**: The company is entirely remote, promoting flexibility and work-life balance.
- **Benefits**: 20 days PTO encourages employees to recharge.
- **Careers**: [Explore TalentHub's Job Openings](https://4dayweek.io/company/talenthub/jobs)

### 2. Forward Together
- **Overview**: A nonprofit dedicated to advocating for the rights of fam